In [10]:
from fredapi import Fred
import warnings
warnings.filterwarnings('ignore')
from fred import Fred
import pandas as pd
import re
import time
from functools import reduce

In [2]:
fr = Fred(api_key='f13196245a409ee3db300071145818eb',response_type='dict')

In [5]:
state_list=["Alabama","Alaska","Arizona","Arkansas","California","Colorado",
  "Connecticut","Delaware",'District of Columbia',"Florida","Georgia","Hawaii","Idaho","Illinois",
  "Indiana","Iowa","Kansas","Kentucky","Louisiana","Maine","Maryland",
  "Massachusetts","Michigan","Minnesota","Mississippi","Missouri","Montana",
  "Nebraska","Nevada","New Hampshire","New Jersey","New Mexico","New York",
  "North Carolina","North Dakota","Ohio","Oklahoma","Oregon","Pennsylvania",
  "Rhode Island","South Carolina","South Dakota","Tennessee","Texas","Utah",
  "Vermont","Virginia","Washington","West Virginia","Wisconsin","Wyoming"]
states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]
states=[x.lower() for x in states]

In [189]:
all_tags=pd.DataFrame(fr.tag.tags())
all_tags['name'][0:50]

In [6]:
tag_list=['commercial','banks','sales']

In [133]:
def get_series_df(series_id,title,frequency,state):
    df=pd.DataFrame(fr.series.observations(series_id))
    df=df.loc[:,['date','value']]
    df.columns=['date',title]
    df['year']=[x.year for x in df['date']]
    if frequency=='A':
        df=df.loc[:,['year',title]]
    if frequency=='Q':
        df['month']=[x.month for x in df['date']]
        df=df.loc[df['month']==1,:]
        df=df.loc[:,['year',title]]
    if frequency=='M':
        df['month']=[x.month for x in df['date']]
        '''
        #Return year data
        df=df.loc[df['month']==1,:]
        df=df.loc[:,['year',title]]
        '''
    df['state']=state
    df=df.loc[df['year']>=1992,:]
    return df

In [409]:
import re
total_df_list1=[]
for tag in tag_list:
    state_count=0
    for state in states:
        temp_search=state+';'+tag
        temp_df=pd.DataFrame(fr.tag.series(temp_search,order_by='popularity'))
        temp_df=temp_df.loc[:,['id','title','frequency_short']]
        temp_df['state']=state
        temp_df['m_id']=[re.sub(state.upper(),'',x) for x in temp_df['id'] ]
        if state_count==0:
            final_df=temp_df
        else:
            final_df=final_df.append(temp_df)
        state_count+=1
    total_df_list1.append(final_df)

In [435]:
scrapy_data_list=[]
for df in total_df_list1:
    get_id=df['m_id'].value_counts()
    get_id=get_id[get_id>=48]
    id_list=list(get_id.index)
    temp_df=df.loc[df['m_id'].isin(id_list)]
    scrapy_data_list.append(temp_df)
scrapy_final = reduce(lambda left,right: left.append(right),scrapy_data_list)
scrapy_final=scrapy_final.reset_index(drop=True)

In [429]:

def merge_id_df(df):
    df=df.reset_index(drop=True)
    state_count=0
    for i in range(df.shape[0]):
        temp_id=df.loc[i,'id']
        temp_df=get_series_df(temp_id,df.loc[i,'m_id'],df.loc[i,'frequency_short'],df.loc[i,'state'])
        #print(temp_df.shape)
        if state_count==0:
            state_df=temp_df
        else:
            state_df=state_df.append(temp_df)
        state_count=state_count+1     
    state_df.columns=['year',df.loc[i,'title'],'state']
    return state_df
    
    

In [414]:
grouped=scrapy_final.groupby('m_id')
group_count=0
for name, group in grouped:
    if group_count==0:
        final_df=merge_id_df(group)
    else:
        temp_df=merge_id_df(group)
        temp_df=temp_df.drop_duplicates()
        final_df=pd.merge(final_df,temp_df,on=['state','year'],how='outer')
        final_df=final_df.drop_duplicates()
    group_count+=1  
final_df.dropna(axis=1,thresh=1200)

    

ALLL
ATRR
COALLL
FFSSP
IBB
IBTEIOA
LLR
NII
NINC
NP
NUM
QATA
QWATA
QWATEC
QWATLLNUI
RALLL
TACAFSS
TAHTA
TAST
TEQC
THTMS
TISBV
TLLFRN
TLLFRPD90
TLLNUI
TNC
TTAST
X08IOMBI
X08IOMBO
X08TDCBMBN
X08TDCBMBT


In [436]:
final_df.shape
final_df.columns=[re.sub('for Wyoming|in Wyoming','',x) for x in final_df.columns]

In [437]:
final_df.head(5)

,year,Allowance for Loan and Lease Losses for Commercial Banks,state,Allocated Transfer Risk Reserves for Commercial Banks,Charge-offs on Allowance for Loan and Lease Losses for Commercial Banks,"Federal Funds Sold and Securities Purchased Under Agreements to Resell in Domestic Offices of the Bank and of Its Edge and Agreement Subsidiaries, and in International Banking Facilities (IBFs) for Commercial Banks (DISCONTINUED)",Interest-Bearing Balances for Commercial Banks,Income (Loss) before Income Taxes and Extraordinary Items and Other Adjustments for Commercial Banks,Loan Loss Reserves for Commercial Banks,Net Interest Income for Commercial Banks,...,Total Investment Securities at Book Value for Commercial Banks (DISCONTINUED),"Total Loan and Lease Finance Receivables, Nonaccrual for Commercial Banks","Total Loan and Lease Finance Receivables, Past Due 90 Days or More and Still Accruing for Commercial Banks","Total Loans and Leases, Net of Unearned Income for Commercial Banks",Total Net Charge-offs for Commercial Banks,Total Trading Assets for Commercial Banks,Interbank Deposits in Commercial Member Banks,Other Deposits in Commercial Member Banks,Deposits in National Commercial Member Banks,Total Deposits in Commercial Member Banks
0,1992,40584.0,ak,0,2462.0,34695.0,48274,30741,40584,56851,...,1952687.0,23235,12045,2013232,1466.0,NaN,NaN,NaN,NaN,NaN
1,1993,40173.0,ak,0,2281.0,54680.0,35699,30801,40173,60031,...,1995240.0,17003,5192,2273294,1195.0,NaN,NaN,NaN,NaN,NaN
2,1994,38471.0,ak,0,2066.0,89130.0,6223,27812,38471,61278,...,NaN,16218,8462,2632478,781.0,0.0,NaN,NaN,NaN,NaN
3,1995,42582.0,ak,0,1856.0,43834.0,9901,23738,42582,61814,...,NaN,15560,6108,2837095,-3259.0,0.0,NaN,NaN,NaN,NaN
4,1996,46840.0,ak,0,6115.0,25364.0,9432,27842,46840,68359,...,NaN,21685,8927,3121934,4667.0,0.0,NaN,NaN,NaN,NaN


In [355]:
final_df.shape

(1404, 32)

In [185]:
temp_df=pd.DataFrame(fr.tag.series('ca;sales',order_by='popularity'))
temp_df=temp_df.loc[:,['id','title']]
temp_df['id']=[re.sub('CA','',x) for x in temp_df['id'] ]
temp_df1=pd.DataFrame(fr.tag.series('ak;sales',order_by='popularity'))
temp_df1['id']=[re.sub('AK','',x) for x in temp_df1['id'] ]
m_df=pd.merge(temp_df,temp_df1,on='id',how='inner')

In [64]:
'''
total_df_list=[]
for tag in tag_list:
    state_count=0
    for state in states:
        temp_search=state+';'+tag
        #print(temp_search)
        temp_df=pd.DataFrame(fr.tag.series(temp_search,order_by='popularity'))
        temp_df=temp_df.loc[:,['id']]
        temp_df['m_id']=[re.sub(state.upper(),'',x) for x in temp_df['id'] ]
        if state_count==0:
            final_df=temp_df
        else:
            final_df=pd.merge(final_df,temp_df,on='m_id',how='inner')
        state_count+=1
    total_df_list.append(final_df)
'''


'''
albama=fr.category.series('27282',order_by='popularity',sort_order='desc')
alska=fr.category.series('27286',order_by='popularity',sort_order='desc')
#help(fr.category.series)
albama=pd.DataFrame(albama)
alska=pd.DataFrame(alska)
albama=albama.loc[:,['id','title']]
alska=alska.loc[:,['id','title']]
import re
albama['title']=[re.sub('for Alabama|in Alabama','',x) for x in albama['title'] ]
alska['title']=[re.sub('for California|in California','',x) for x in alska['title'] ]
albama['id_me']=[re.sub('AL','',x) for x in albama['id'] ]
alska['id_me']=[re.sub('CA','',x) for x in alska['id'] ]
'''

"\nalbama=fr.category.series('27282',order_by='popularity',sort_order='desc')\nalska=fr.category.series('27286',order_by='popularity',sort_order='desc')\n#help(fr.category.series)\nalbama=pd.DataFrame(albama)\nalska=pd.DataFrame(alska)\nalbama=albama.loc[:,['id','title']]\nalska=alska.loc[:,['id','title']]\nimport re\nalbama['title']=[re.sub('for Alabama|in Alabama','',x) for x in albama['title'] ]\nalska['title']=[re.sub('for California|in California','',x) for x in alska['title'] ]\nalbama['id_me']=[re.sub('AL','',x) for x in albama['id'] ]\nalska['id_me']=[re.sub('CA','',x) for x in alska['id'] ]\n"

In [364]:
from census import Census


https://api.census.gov/data/2017/acs/acs1?get=B00001_001E,NAME&for=state:*&time=2014*&key=879116b83035eef6bc66700515158f25b5f38c57

In [365]:
c = Census("879116b83035eef6bc66700515158f25b5f38c57")

In [ ]:
https://api.census.gov/data/timeseries/bds/firms?get=estabs,sic1&for=state:*&time=2014&key=879116b83035eef6bc66700515158f25b5f38c57

In [402]:
html='''https://api.census.gov/data/timeseries/bds/firms?get=estabs,sic1&for=state:*&time=2014&key=879116b83035eef6bc66700515158f25b5f38c57'''

In [367]:
import requests

In [400]:
response=requests.get(html)

In [372]:
import pandas as pd
import cenpy as cen
import pysal

In [376]:
datasets = list(cen.explorer.available(verbose=True).items())
datasets=pd.DataFrame(datasets)

In [442]:
'a' in 'abc'

True

In [2]:
'''
for i in datasets.iloc[:,1]:
    temp_str=i.lower()
    if 'business' in temp_str:
        print(temp_str)
'''

"\nfor i in datasets.iloc[:,1]:\n    temp_str=i.lower()\n    if 'business' in temp_str:\n        print(temp_str)\n"

In [ ]:
for i in datasets.iloc[:,0]:
    if i[0:3]=='BDS':
        print(i)
dataset = 'BDSFirmsTimeSeries'
con = cen.base.Connection(dataset)
con.geographies['fips']
con.query(cols=['estabs'],geo_unit='state',apikey='879116b83035eef6bc66700515158f25b5f38c57')
state_category=pd.DataFrame(fr.category.children('27281'))
state_category=state_category.loc[state_category['name'].isin(state_list),:]
state_category.reset_index(drop=True)